# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import itertools
from tabulate import tabulate

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject
from inauguralproject import HouseholdSpecializationModelClass
# Set parameters

rho = 2
nu = 0.001
epsilon = 1
omega = 0.5
alpha = 0.5
sigma = 1
wM = 1
wF = 1
beta0_target = 0.4
beta1_target = -0.1


# Question 1

**Explain how you solve the model**

In [11]:
# a. Solving the model

OurModel = HouseholdSpecializationModelClass()

arsigma=np.array([0.5,1,1.5]) 

aralpha=np.array([0.25,0.5,0.75])

result2=[]
for k,v in itertools.product(arsigma,aralpha): #for all combinations in the two arrays
    OurModel.par.sigma=k #values from the first array is used as sigma parameters in the model
    OurModel.par.alpha=v #values from second array used as alpha parameters
    temp=OurModel.solve_discrete() #naming the solution function
    result2.append(temp.g) #appending the solutions of the variable "g"(HF/HM) to the empty list "result2"

print(result2)





c:\Users\juliu\OneDrive\Documents\projects-2023-bulen\inauguralproject\inauguralproject.py:79: RuntimeWarning: divide by zero encountered in reciprocal
  H = ((1-par.alpha)*HM**((par.sigma-1)/par.sigma) + par.alpha*HF**((par.sigma-1)/par.sigma))**(par.sigma/(par.sigma-1))


[0.5454545454545454, 1.0, 1.8333333333333333, 0.3076923076923077, 1.0, 3.25, 0.21428571428571427, 1.0, 4.666666666666667]


In [12]:

# b. Illustrating results

# i. Changing the solution matrix from 9x1 list to 3x3 dataframe and adding alpha and sigma as rows and columns
ser=pd.Series(result2)
ser2=ser.values.reshape((3,3))
df=pd.DataFrame(ser2, index=[value], columns=[value1])

# ii. Style
s=pd.concat( #creating a dataframe with concatenated row and column titles, and saving it as "s"
    [pd.concat(
    [df],
    keys=["alpha"], axis=1)],
    keys=["sigma"])

cm = sns.light_palette("blue", as_cmap=True) #creating and naming a color palette
s.style.\
    set_caption("HF/HM").\
    background_gradient(cmap=cm, axis=None).\
    format_index(axis=0,precision=2).\
    format_index(axis=1, precision=2).\
    format(precision=4) #styling the "s" dataframe with caption, background color gradient, and formatting decimals









# Question 2

Explain your code and procedure

In [3]:
# code





# Question 3

Explain your code and procedure

In [4]:
# code

ADD CONCISE CONLUSION.